In [37]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import torch

device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/hansol/.cache/huggingface/token
Login successful


In [9]:
from datasets import load_dataset

korquad = load_dataset("Hansollll/korquad_v1")

Using custom data configuration Hansollll--korquad_v1-103f86c4266af435


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/12082 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/48325 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /Users/hansol/.cache/huggingface/datasets/Hansollll___parquet/Hansollll--korquad_v1-103f86c4266af435/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
korquad

DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 12082
    })
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 48325
    })
})

In [14]:
from transformers import AutoTokenizer
    

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [15]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"]
        end_char = answer["answer_start"] + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [28]:
tokenized_korquad = korquad.map(preprocess_function, batched=True, remove_columns=korquad["train"].column_names)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/49 [00:00<?, ?ba/s]

In [29]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [30]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

loading configuration file config.json from cache at /Users/hansol/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /Users/hansol/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were 

In [31]:
model.to(device)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

In [34]:
training_args = TrainingArguments(
    output_dir="qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_korquad["train"],
    eval_dataset=tokenized_korquad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/Users/hansol/Desktop/KorQuAD/qa_model is already a clone of https://huggingface.co/Hansollll/qa_model. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [38]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [39]:
trainer.train()

***** Running training *****
  Num examples = 48325
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9063
  Number of trainable parameters = 66364418


Epoch,Training Loss,Validation Loss
1,1.704200,1.589765
2,1.495500,1.458980
3,1.327400,1.390601


Saving model checkpoint to qa_model/checkpoint-500
Configuration saved in qa_model/checkpoint-500/config.json
Model weights saved in qa_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in qa_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in qa_model/checkpoint-500/special_tokens_map.json
tokenizer config file saved in qa_model/tokenizer_config.json
Special tokens file saved in qa_model/special_tokens_map.json
Saving model checkpoint to qa_model/checkpoint-1000
Configuration saved in qa_model/checkpoint-1000/config.json
Model weights saved in qa_model/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in qa_model/checkpoint-1000/tokenizer_config.json
Special tokens file saved in qa_model/checkpoint-1000/special_tokens_map.json
tokenizer config file saved in qa_model/tokenizer_config.json
Special tokens file saved in qa_model/special_tokens_map.json
Saving model checkpoint to qa_model/checkpoint-1500
Configuration saved in qa_model/check

tokenizer config file saved in qa_model/tokenizer_config.json
Special tokens file saved in qa_model/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 12082
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9063, training_loss=1.6671586973730543, metrics={'train_runtime': 36497.6571, 'train_samples_per_second': 3.972, 'train_steps_per_second': 0.248, 'total_flos': 1.4765792763391488e+16, 'train_loss': 1.6671586973730543, 'epoch': 3.0})

In [47]:
question

'장 폴 고티에가 아동복 제작을 위해 스케치를 그린 나이는?'

In [49]:
context

"15세에 그는 아동복 제작을 위한 스케치를 그린다. 모든 의상이 마르셀 로샤(Marcel Rochas)에 의해 고안된 자크 베커(Jacques Becker)의 영화 '팔발라(Falbalas)'를 본 후에, 어린 장 폴 고티에는 디자이너가 되기로 결심한다. 그는 이브 생로랑(Yves Saint Laurent)에게 자신의 스케치들을 보내지만 거부당한다. (이브 생 로랑은 그가 선택한 색이 지나치게 밝다고 생각했다.) 그의 스케치는 피에르 가르뎅에게로 보내지고 18살이 되던 날에 피에르 가르뎅의 의상실에 견습생으로 들어가 1년이 좀 덜 되게 일한다. 1970년 11월 13일 ORTF에서 방영된 방송을 피에르 가르뎅과 함께 촬영하기도 했다."

In [42]:
question = korquad['test'][-2]['question']
context = korquad['test'][-2]['context']

In [43]:
inputs = tokenizer(question, context, return_tensors="pt")

In [57]:
with torch.no_grad():
    outputs = model(**inputs)

In [58]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [59]:
print(answer_start_index, answer_end_index)

tensor(55) tensor(56)


In [60]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'15ᄉ'